We have spent some time exploring the space of finding the best answer to a set of particular questions and have delved deeper into this problem space by attempting to find ways of breaking answers down into discretely composable solution parts.

This brings us to a new problem: What if the user (herself) doesn't actually know what she wants? How might we improve the question for the user if hers is not sufficiently detailed or formulated?

## Proposed approach:
- evaluate the question
- determine what action(s) might be taken to address the question
- if the gathered information produces a broad variety of results, we might determine that more detail should be provided by the user (or by some form of deduction).
- if we cannot automatically deduce what elaboration the user might benefit from, provide guidance to the user for how to formulate a better question.

## Example
User: "How do I calculate tax"
Assistant: 
1. run search
2. found three results
3. evaluation: results vary broadly
4. trigger elaboration attempt
"Are you trying to calculate sales tax?"


In [2]:
import sys
sys.path.append("../")
from fns import get_embeddings, messages_prompt, get_intent
from tools import search as vector_search, page_scraper, pick

In [3]:
import json

def essentials(results):
    return [result["content"] for result in results["value"]]

def prompt(results):
    return f"""
You are a helpful assistant. Your job is to determine if a user's question could
use better phrasing. If so, you should suggest a better way to phrase the
question.
 
You will have both the user's question and a list of search results based on the
question. You should use the search results to determine if the user's question
could be improved. If the search results vary significantly in their
content, you should suggest a better question. If not, you pick one of the
search results to answer the user's question.

If you choose to suggest a better question return the phrasing you suggest. No 
additional context is needed.

Here are the search results: {json.dumps(essentials(results), indent=2)}
"""

def query(question):
    embed = get_embeddings(question)
    results = vector_search(embeddings=embed)
    #print(results)
    messages = [{
        "role": "system",
        "content": prompt(results)
    },
    {
        "role": "user",
        "content": question
    },
    #{
    #    "role": "assistant",
    #    "content": json.dumps(results)
    #},
    ]
    final = messages_prompt(messages)
    return final

In [4]:
res1 = query("How do you calculate tax in the state of PA?")
print(res1)

Your question could be more specific to get the most accurate information. You might want to ask, "How do I look up tax rates by jurisdiction in Pennsylvania?" or "How is the Pennsylvania Local Services Tax calculated for payroll?"


In [5]:
res2 = query("How do you calculate sales tax in the state of PA?")
print(res2)

Based on your question, it seems you are interested in the process of calculating sales tax in Pennsylvania. However, the search results provided do not directly answer this question. They discuss looking up tax rates by jurisdiction or Tax Area ID and payroll taxes but do not provide a method for calculating sales tax. 

A better phrasing might be: "What is the process to look up sales tax rates in Pennsylvania?"


In [6]:
res3 = query("What are the features of O Series?")
print(res3)

Your question is quite broad and could be interpreted in many ways. To get more specific information, you might want to ask: "What are the key components and functionalities of the O Series On-Premise technology?"
